In [1]:
import sys, os
#sys.path.insert(0, os.path.dirname(os.path.abspath('../desdeo_emo')))
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [2]:
from desdeo_problem.Problem import DataProblem

from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_emo.othertools.ReferenceVectors import ReferenceVectors
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor


from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_

In [3]:
from pyDOE import lhs
from sklearn.gaussian_process.kernels import Matern
from pygmo import fast_non_dominated_sorting as nds

In [4]:
import baseADM
from baseADM import *
import generatePreference as gp


In [5]:
problem_name = "ZDT1"
prob = test_problem_builder(problem_name)

In [6]:
x = lhs(30, 200)
y = prob.evaluate(x)

x_names = [f'x{i}' for i in range(1,31)]
y_names = ["f1", "f2"]

data = pd.DataFrame(np.hstack((x,y.objectives)), columns=x_names+y_names)

In [7]:
problem = DataProblem(data=data, variable_names=x_names, objective_names=y_names)

In [8]:
problem.train(GaussianProcessRegressor, model_parameters={"kernel": Matern(nu=3/2)})

In [9]:
evolver = NSGAIII(problem,
                  use_surrogates=True,
                  n_iterations=3,
                  n_gen_per_iter=50)
evolver2 = RVEA(problem,
                use_surrogates=True,    
                n_iterations=3,
                n_gen_per_iter=50)

In [10]:
plot, pref = evolver.requests()
plot2, pref2 = evolver2.requests()

In [11]:
pref
pref2

In [12]:
while evolver.continue_evolution():
    evolver.iterate()
while evolver2.continue_evolution():
    evolver2.iterate()

In [13]:
objectives = evolver.population.objectives
objectives2 = evolver2.population.objectives

In [14]:
cf = generate_composite_front(objectives, objectives2)
fig = go.Figure(data=go.Scatter(x=cf[:,0],
                                y=cf[:,1],
                                   mode="markers",
                                   marker_size=3))
fig

In [15]:
lattice_resolution = 4
reference_vectors = ReferenceVectors(lattice_resolution, problem.n_of_objectives)

In [16]:
base = baseADM(cf, reference_vectors)
base2 = baseADM(objectives, reference_vectors)

In [17]:
visualize_2D_front_rvs(base.normalized_front, base2.vectors)

In [18]:
rp = gp.generateRP4learning(base)
visualize_2D_front_rp(base.composite_front, rp)

In [19]:
max_assigned_vector = gp.get_max_assigned_vector(base.assigned_vectors)
rp2 = gp.generateRP4decision(base, max_assigned_vector)
visualize_2D_front_rp(base.composite_front, rp2)

In [32]:
from pymoo.factory import get_problem
from pymoo.performance_indicator.rmetric import RMetric
#from rmetric import RMetric

problemR = get_problem("zdt1")
ref_point = rp2.reshape(1, 2)
#r_metric = RMetric(objectives, cf, rp2, p)
rmetric = RMetric(problemR, ref_point)
rigd, rhv = rmetric.calc(objectives, others = cf)
print(f"R-IGD: {rigd}, R-HV: {rhv}")





R-IGD: 0.46300304750091786, R-HV: 3.8854664817639435
